In [1]:
import pandas as p
import numpy 
from matplotlib_venn import venn2
from matplotlib import pyplot as plt
import re

In [2]:
human_high = p.read_csv("/Users/admin/Documents/Mouse/Annotation_tables/human_high.csv",header=0)
human_medium = p.read_csv("/Users/admin/Documents/Mouse/Annotation_tables/human_medium.csv",header=0)
mouse_high = p.read_csv("/Users/admin/Documents/Mouse/Annotation_tables/mouse_high.csv",header=0)
mouse_medium = p.read_csv("/Users/admin/Documents/Mouse/Annotation_tables/mouse_medium.csv",header=0)

In [3]:
def filter_network(networks,outfile):
    banned_db = set(["biogrid","intact","microT","miRanda","miRBase","miRecords",
                 "PicTar","string","miRTarBase","TargetScan","transmir"])
    out = open(outfile,"w")
    for network in networks:
        for i in network.index:
            databases = set((network.loc[i,"database"]).split(","))
            if len(databases&banned_db) == 0:
                line = [str(word) for word in network.loc[i]]
                print >> out,"\t".join(line)
    out.close()


In [4]:
filter_network([human_high,human_medium],"/Users/admin/Documents/Mouse/Annotation_tables/human_RN.txt")
filter_network([mouse_high,mouse_medium],"/Users/admin/Documents/Mouse/Annotation_tables/mouse_RN.txt")

#Obtain Transcription factors

In [2]:
import re
TF_list = open('/Users/admin/Documents/Mouse/Dorothea/T_FACTORS_names.txt',"r")
TF_set = []
for line in TF_list:
    line = re.sub("\n","",line)
    TFs = re.split("::",line)
    for TF in TFs: TF_set.append(TF)
TF_list.close()
TF_set = set(TF_set)
TF_set = list(TF_set)
TF_set = [TF.lower() for TF in TF_set]

#Read human RN and look for the TFs in the network

In [3]:
def read_RN(route):
    RN = {}
    import re
    RN_file = open(route,"r")
    for line in RN_file:
        components = re.split("\t",line)
        Regulator = (components[0]).lower()
        Regulated = (components[2]).lower()
        if Regulator in RN:
            (RN[Regulator]).append(Regulated)
        else:
            RN[Regulator]=[Regulated]
    RN_file.close()
    return RN


In [4]:
Human_RN = read_RN("/Users/admin/Documents/Mouse/Annotation_tables/human_RN.txt")

In [5]:
print len(Human_RN)
print len(TF_set)

305
153


In [6]:
Present = []
Absent = []
for TF in TF_set:
    if TF in Human_RN:
        Present.append(TF)
    else:
        Absent.append(TF)
print len(Present)
print len(Absent)

H_Pres = list(Present)
H_Abs = list(Absent)

67
86


#Obtaining Orthologs for mouse analysis, and comparison measuring the real absence

In [7]:
import re
ortholog_file = open("/Users/admin/Documents/Mouse/Annotation_tables/human_mouse_ort.txt","r")
orthologs = {}
for line in ortholog_file:
    line = re.sub("\n","",line)
    components = re.split("\t",line)
    orthologs[(components[0]).lower()] = (components[1]).lower()

In [8]:
Present = []
Absent = []
for TF in TF_set:
    if TF in orthologs:
        Present.append(TF)
    else:
        Absent.append(TF)
print len(Present)
print len(Absent)
print ",".join(Absent)

137
16
ap1,rora_1,rora_2,myf,rxr,mizf,nf-kappab,zfp423,mzf1_5-13,tcfcp2l1,mzf1_1-4,ewsr1-fli1,evi1,tcfe2a,gfi,rar_dr5


#Using synonyms no increase the amount of TFs

In [9]:
synonyms = ["jun","rora","myod1","myf5","nr1h4","nfkb1","znf423","mzf1",
            "ewsr1","mecom","tcf3","gfi1","rara","tnfrsf10b"]

TF_set = list(TF_set)
TF_set_syn = set(TF_set + synonyms)

Present = []
Absent = []
for TF in TF_set_syn:
    if TF in Human_RN:
        Present.append(TF)
    else:
        Absent.append(TF)

H_pres = list(Present)       
print len(Present)
print len(Absent)
print len(TF_set)



70
94
153


#which synonyms were used?

In [10]:
syn = [gene for gene in H_pres if gene in synonyms]
print syn
dor_syn = ["rar_dr5","ap1","nf-kappab","rora_1","rora_2","evi1"]

['nfkb1', 'rara', 'rora', 'jun', 'mecom']


In [11]:
import re

TF_list = open('/Users/admin/Documents/Mouse/Dorothea/T_FACTORS_names.txt',"r")
TF_set = []
for line in TF_list:
    line = re.sub("\n","",line)
    TFs = re.split("::",line)
    for TF in TFs: TF_set.append(TF)
TF_list.close()
TF_set = set(TF_set)
TF_set = list(TF_set)
TF_set = [TF.lower() for TF in TF_set]


TF_set = [TF for TF in TF_set if TF not in dor_syn]

TF_set = set(TF_set + syn)
print len(TF_set)



150


In [12]:
Present = []
Absent = []
for TF in TF_set:
    if TF in Human_RN:
        Present.append(TF)
    else:
        Absent.append(TF)
print len(Present)
print len(Absent)

H_Pres = list(Present)
H_Abs = list(Absent)

70
80


In [29]:
set1 = set(TF_set)
set2 = set(H_Pres)

venn2([set1, set2], ('DoRothea', 'RegNetwork Human'))
plt.show()

#Obtaining TFs present in Mouse RN

In [13]:
mouse_orthologs = [orthologs[key] for key in orthologs if (key in TF_set) and (orthologs[key]!="")]
print len(mouse_orthologs)

137


In [14]:
Mouse_RN = read_RN("/Users/admin/Documents/Mouse/Annotation_tables/mouse_RN.txt")
print len(Mouse_RN)
print len(TF_set)

205
150


In [19]:
Present = []
Absent = []
for TF in mouse_orthologs:
    if TF in Mouse_RN:
        Present.append(TF)
    else:
        Absent.append(TF)
print len(Present)
print len(Absent)
M_pres = list(Present)
M_abs = list(Absent)

62
75


In [33]:
H_pres_mouse = [gene for gene in orthologs if (orthologs[gene] in M_pres)]
print len(H_pres_mouse)

62


In [21]:
set1 = set(TF_set)
set2 = set(H_pres_mouse)

venn2([set1, set2], ('DoRothea', 'RegNetwork Mouse'))
plt.show()

In [34]:
set1 = set(H_pres)
set2 = set(H_pres_mouse)

venn2([set1, set2], ('RegNetwork Human', 'RegNetwork Mouse'))
plt.show()

In [22]:
#dor_syn = ["rar_dr5","ap1","nf-kappab","rora_1","rora_2","evi1"]
#['nfkb1', 'rara', 'rora', 'jun', 'mecom']
dor_syn_dic = {"nfkb1":["nf-kappab"], "rara":["rar_dr5"],
               "rora":["rora_1","rora_2"],
               "jun":["ap1"],"mecom":["evi1"]
}

### Reading Dorothea RN

In [30]:
DOR_RN_file = open("/Users/admin/Documents/Mouse/Dorothea/DOR_RN.txt","r") 
DOR_RN = {}
for line in DOR_RN_file:
    line = re.sub("\n","",line)
    components = re.split("\t",line)
    TFs = re.split(":",components[0])
    Regulated = re.split(",",components[1])
    for TF in TFs:
        DOR_RN[TF.lower()] = Regulated
DOR_RN_file.close()

In [40]:
DOR_RN

{'': ['KARSP1',
  'LINC00486',
  'UQCRFS1P3',
  'SPACA7',
  'MIR141',
  'NFXL1',
  'ENC1',
  'SALL1',
  'GABRP',
  'KRT8P43',
  'STT3A',
  'RPS4XP16',
  'CRISPLD2',
  'HERC5',
  'SOAT1',
  'TRDD2',
  'SLC19A2',
  'UCN',
  'C9orf152',
  'ZNF792',
  'KIF21B',
  'PWWP2A',
  'PANX3',
  'FAM104A',
  'C17orf80',
  'FABP7P1',
  'CRYZL1',
  'TGIF2P1',
  'RPL7P22',
  'TMEM87B',
  'VWA7',
  'CDKL2',
  'KCNAB1',
  'PUSL1',
  'BCRP3',
  'GGT5',
  'LRRN4',
  'HLA-DPA2',
  'HERC3',
  'ANAPC10P1',
  'C11orf40',
  'LINC00330',
  'NXT2',
  'URB1',
  'ELL2P3',
  'NYX',
  'SLC23A1',
  'TAF7L',
  'MESP1',
  'RPS7P3',
  'MIR1231',
  'SRSF10',
  'GCHFR',
  'NEUROD2',
  'PAX3',
  'FAM53B',
  'METTL10',
  'FGF1',
  'DMRT2',
  'FBXO10',
  'NIPA2P2',
  'DUSP13',
  'LINC00113',
  'MBLAC1',
  'AKAP14',
  'PLS3',
  'MAPKAPK3',
  'UPP2',
  'XKRYP3',
  'TRIM26BP',
  'PROZ',
  'PSMD4P1',
  'AGBL5',
  'XKRX',
  'NISCH',
  'TNNC1',
  'NFIL3',
  'CES1',
  'ATP6V0D1',
  'POT1',
  'MIR33A',
  'UQCRC2',
  'ZNF775',
  'CXCR

In [44]:
H_pres_mouse = [gene for gene in H_pres_mouse if gene in Human_RN and gene in TF_set]
print len(H_pres_mouse)

58


In [48]:
HUMAN_RG = []
HUMAN_DOR = []
MOUSE_RG = []
IDs = []
for TF in H_pres_mouse:
    IDs.append(TF)
    HUMAN_RG.append(len(Human_RN[TF]))
    MOUSE_RG.append(len(Mouse_RN[orthologs[TF]]))
    if TF in dor_syn_dic:
        print "combination"
        count = 0
        for gene in dor_syn_dic[TF]:
            count=len(DOR_RN[gene]) + count
        HUMAN_DOR.append(count)
    else:
        HUMAN_DOR.append(len(DOR_RN[TF]))
        
        

combination
combination
combination


In [51]:
data = {"HUMAN_DOR":HUMAN_DOR,"HUMAN_RG":HUMAN_RG,"MOUSE_RG":MOUSE_RG}
data = p.DataFrame(data, index=IDs)

In [60]:
data = data.sort(columns="HUMAN_DOR", ascending=False, inplace=False)

In [64]:
data.plot(kind='bar')
plt.show()

In [67]:
data[0:25].plot(kind='bar')
plt.show()

In [68]:
data[27:58].plot(kind='bar')
plt.show()